In [1]:
# Clone your GitHub repo (you’ll be prompted to authorize if it's private)
!git clone https://github.com/colterwood/LHL-final-final-project.git

Cloning into 'LHL-final-final-project'...
remote: Enumerating objects: 154, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 154 (delta 75), reused 22 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (154/154), 1.52 MiB | 2.27 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [46]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from google.colab import files

In [3]:
# load the player game logs CSV from the data folder
df = pd.read_csv("LHL-final-final-project/data/wnba_model_ready_v2.csv")


# preview
df.head()

,advanced_3par,advanced_ast_pct,advanced_blk_pct,advanced_drtg,advanced_ftr,advanced_ortg,advanced_pace,advanced_pace_by_team_day_of_week,advanced_stl_pct,advanced_trb_pct,...,team_3p_pct,team_day_median_allowed,team_ft_pct,team_vs_opp_homeaway_median_score,team_vs_opp_median_score,win_loss,team,opp,team_score,opp_score
0,0.254,58.8,22.0,100.4,0.284,114.0,80.7,79.4,9.9,49.3,...,0.471,78.0,0.842,86.0,87.0,1,ATL,LAS,92,81
1,0.348,77.8,11.6,107.3,0.409,103.6,82.0,80.2,9.8,46.5,...,0.435,84.0,0.778,75.5,76.0,2,ATL,PHO,85,88
2,0.306,56.7,5.6,100.5,0.306,107.0,77.6,74.8,11.6,50.7,...,0.318,77.0,0.727,95.0,83.0,1,ATL,DAL,83,78
3,0.324,74.2,3.0,124.6,0.206,107.0,73.8,76.1,12.2,50.8,...,0.273,80.0,0.786,71.5,71.5,2,ATL,MIN,79,92
4,0.339,73.1,4.3,84.4,0.339,92.0,79.4,79.4,11.3,53.7,...,0.286,78.0,0.714,74.5,71.0,1,ATL,WAS,73,67


In [7]:
for col in df.columns:
  print(col)

advanced_3par
advanced_ast_pct
advanced_blk_pct
advanced_drtg
advanced_ftr
advanced_ortg
advanced_pace
advanced_pace_by_team_day_of_week
advanced_stl_pct
advanced_trb_pct
advanced_ts_pct
day
defensive_four_factors_drb_pct
defensive_four_factors_efg_pct
defensive_four_factors_ft_per_fga
defensive_four_factors_tov_pct
month
month_by_team_day_of_week
offensive_four_factors_orb_pct
offensive_four_factors_tov_pct
opponent_3p_pct
opponent_fg_pct
opponent_ft_pct
team_3p_pct
team_day_median_allowed
team_ft_pct
team_vs_opp_homeaway_median_score
team_vs_opp_median_score
win_loss
team
opp
team_score
opp_score


In [30]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# targets
target_cols = ["team_score", "opp_score"]

# models to train: 12 teams + league-wide
teams = df["team"].unique().tolist() + ["League"]

# store overall results summary
results = []

# store row-level predictions
all_predictions = []

for team_name in teams:
    # subset data
    if team_name == "League":
        df_team = df.copy()
    else:
        df_team = df[df["team"] == team_name].copy()

    # features: drop non-feature columns
    X = df_team.drop(columns=["team", "opp", "team_score", "opp_score"])
    y = df_team[["team_score", "opp_score"]]

    # train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # track team + opp info from original df
    meta = df_team.loc[y_test.index, ["team", "opp", "team_score", "opp_score"]].reset_index(drop=True)

    # model
    model = MultiOutputRegressor(RandomForestRegressor(random_state=42))
    model.fit(X_train, y_train)

    # predict
    y_pred = model.predict(X_test)

    # compute row-level absolute errors
    team_score_mae = np.abs(y_pred[:, 0] - meta["team_score"].values)
    opp_score_mae = np.abs(y_pred[:, 1] - meta["opp_score"].values)

    # build prediction rows
    for i in range(len(meta)):
        all_predictions.append({
            "Model": team_name,
            "team": meta.loc[i, "team"],
            "opp": meta.loc[i, "opp"],
            "team_score": meta.loc[i, "team_score"],
            "team_score_pred": y_pred[i, 0],
            "team_score_mae": team_score_mae[i],
            "opp_score": meta.loc[i, "opp_score"],
            "opp_score_pred": y_pred[i, 1],
            "opp_score_mae": opp_score_mae[i],
        })

    # overall summary metrics
    for i, col in enumerate(target_cols):
        mae = mean_absolute_error(y_test.iloc[:, i], y_pred[:, i])
        mse = mean_squared_error(y_test.iloc[:, i], y_pred[:, i])
        rmse = np.sqrt(mse)
        results.append({
            "Model": team_name,
            "Target": col,
            "MAE": mae,
            "MSE": mse,
            "RMSE": rmse
        })

# create final prediction DataFrame
predictions_df = pd.DataFrame(all_predictions)

# preview
predictions_df.head()

,Model,team,opp,team_score,team_score_pred,team_score_mae,opp_score,opp_score_pred,opp_score_mae
0,ATL,ATL,CON,67,80.73,13.73,80,88.80,8.80
1,ATL,ATL,NYL,75,80.31,5.31,81,85.01,4.01
2,ATL,ATL,CON,78,83.13,5.13,74,73.77,0.23
3,ATL,ATL,PHO,72,71.97,0.03,63,70.26,7.26
4,ATL,ATL,WAS,73,79.47,6.47,67,73.95,6.95


In [31]:
predictions_df[predictions_df["Model"] == "ATL"]

,Model,team,opp,team_score,team_score_pred,team_score_mae,opp_score,opp_score_pred,opp_score_mae
0,ATL,ATL,CON,67,80.73,13.73,80,88.80,8.80
1,ATL,ATL,NYL,75,80.31,5.31,81,85.01,4.01
2,ATL,ATL,CON,78,83.13,5.13,74,73.77,0.23
3,ATL,ATL,PHO,72,71.97,0.03,63,70.26,7.26
4,ATL,ATL,WAS,73,79.47,6.47,67,73.95,6.95
5,ATL,ATL,MIN,55,62.88,7.88,68,73.50,5.50
6,ATL,ATL,WAS,76,80.76,4.76,73,70.27,2.73
7,ATL,ATL,PHO,80,83.77,3.77,82,88.26,6.26


In [32]:
predictions_df[predictions_df["Model"] == "CHI"]

,Model,team,opp,team_score,team_score_pred,team_score_mae,opp_score,opp_score_pred,opp_score_mae
8,CHI,CHI,SEA,71,72.71,1.71,84,85.00,1.00
9,CHI,CHI,MIN,62,67.42,5.42,70,72.65,2.65
10,CHI,CHI,LVA,83,84.75,1.75,95,87.01,7.99
11,CHI,CHI,PHO,68,66.72,1.28,86,84.11,1.89
12,CHI,CHI,SEA,68,69.58,1.58,77,73.56,3.44
13,CHI,CHI,IND,83,80.11,2.89,91,86.74,4.26
14,CHI,CHI,PHO,88,82.09,5.91,93,90.23,2.77
15,CHI,CHI,CON,80,76.62,3.38,82,83.63,1.63


In [33]:
predictions_df[predictions_df["Model"] == "CON"]

,Model,team,opp,team_score,team_score_pred,team_score_mae,opp_score,opp_score_pred,opp_score_mae
16,CON,CON,MIN,78,79.55,1.55,73,73.89,0.89
17,CON,CON,WAS,94,84.60,9.40,91,82.28,8.72
18,CON,CON,SEA,61,68.54,7.54,72,72.43,0.43
19,CON,CON,LAS,69,69.87,0.87,61,63.30,2.30
20,CON,CON,CHI,86,89.18,3.18,82,79.43,2.57
21,CON,CON,DAL,85,88.90,3.90,67,67.66,0.66
22,CON,CON,LVA,71,76.00,5.00,84,77.85,6.15
23,CON,CON,CHI,82,75.97,6.03,80,76.20,3.80


In [34]:
predictions_df[predictions_df["Model"] == "DAL"]

,Model,team,opp,team_score,team_score_pred,team_score_mae,opp_score,opp_score_pred,opp_score_mae
24,DAL,DAL,PHO,96,97.86,1.86,104,104.97,0.97
25,DAL,DAL,MIN,94,95.34,1.34,88,92.74,4.74
26,DAL,DAL,WAS,84,85.60,1.60,92,97.58,5.58
27,DAL,DAL,NYL,74,71.15,2.85,94,93.04,0.96
28,DAL,DAL,LAS,84,82.77,1.23,83,85.47,2.47
29,DAL,DAL,MIN,78,78.41,0.41,90,90.67,0.67
30,DAL,DAL,SEA,81,77.46,3.54,83,83.63,0.63
31,DAL,DAL,NYL,71,70.80,0.20,79,82.80,3.80


In [35]:
predictions_df[predictions_df["Model"] == "IND"]

,Model,team,opp,team_score,team_score_pred,team_score_mae,opp_score,opp_score_pred,opp_score_mae
32,IND,IND,PHO,88,89.37,1.37,82,81.20,0.80
33,IND,IND,ATL,91,91.47,0.47,79,80.62,1.62
34,IND,IND,WAS,88,89.79,1.79,81,84.65,3.65
35,IND,IND,PHO,98,95.37,2.63,89,88.65,0.35
36,IND,IND,SEA,83,84.62,1.62,85,89.76,4.76
37,IND,IND,CON,72,80.40,8.40,89,98.13,9.13
38,IND,IND,LVA,74,72.25,1.75,78,80.19,2.19
39,IND,IND,SEA,92,89.96,2.04,75,79.36,4.36


In [36]:
predictions_df[predictions_df["Model"] == "LAS"]

,Model,team,opp,team_score,team_score_pred,team_score_mae,opp_score,opp_score_pred,opp_score_mae
40,LAS,LAS,LVA,98,90.17,7.83,93,87.47,5.53
41,LAS,LAS,NYL,88,89.15,1.15,98,95.98,2.02
42,LAS,LAS,NYL,80,82.45,2.45,93,90.42,2.58
43,LAS,LAS,LVA,71,70.26,0.74,87,83.17,3.83
44,LAS,LAS,DAL,83,84.11,1.11,84,85.84,1.84
45,LAS,LAS,MIN,76,70.39,5.61,81,80.19,0.81
46,LAS,LAS,SEA,87,83.40,3.60,90,87.63,2.37
47,LAS,LAS,CON,61,68.82,7.82,69,70.55,1.55


In [37]:
predictions_df[predictions_df["Model"] == "LVA"]

,Model,team,opp,team_score,team_score_pred,team_score_mae,opp_score,opp_score_pred,opp_score_mae
48,LVA,LVA,DAL,104,96.67,7.33,85,78.30,6.70
49,LVA,LVA,IND,88,89.82,1.82,69,74.65,5.65
50,LVA,LVA,WAS,88,88.55,0.55,77,78.73,1.73
51,LVA,LVA,MIN,87,93.20,6.20,98,95.83,2.17
52,LVA,LVA,MIN,80,79.43,0.57,66,75.98,9.98
53,LVA,LVA,SEA,94,85.45,8.55,83,79.02,3.98
54,LVA,LVA,CON,84,84.44,0.44,71,73.92,2.92
55,LVA,LVA,MIN,74,78.57,4.57,87,93.91,6.91


In [38]:
predictions_df[predictions_df["Model"] == "MIN"]

,Model,team,opp,team_score,team_score_pred,team_score_mae,opp_score,opp_score_pred,opp_score_mae
56,MIN,MIN,CON,73,81.69,8.69,78,79.93,1.93
57,MIN,MIN,DAL,88,88.66,0.66,94,83.42,10.58
58,MIN,MIN,PHO,73,80.28,7.28,60,63.78,3.78
59,MIN,MIN,WAS,99,89.41,9.59,83,79.35,3.65
60,MIN,MIN,ATL,92,90.00,2.00,79,80.97,1.97
61,MIN,MIN,LAS,81,84.08,3.08,76,78.68,2.68
62,MIN,MIN,NYL,88,87.42,0.58,79,76.94,2.06
63,MIN,MIN,LVA,98,92.05,5.95,87,83.09,3.91


In [40]:
predictions_df[predictions_df["Model"] == "NYL"]

,Model,team,opp,team_score,team_score_pred,team_score_mae,opp_score,opp_score_pred,opp_score_mae
64,NYL,NYL,MIN,76,76.23,0.23,67,69.80,2.80
65,NYL,NYL,LAS,98,98.87,0.87,88,85.94,2.06
66,NYL,NYL,LAS,93,93.04,0.04,80,78.94,1.06
67,NYL,NYL,LVA,79,78.45,0.55,67,69.05,2.05
68,NYL,NYL,CHI,81,81.02,0.02,90,84.96,5.04
69,NYL,NYL,WAS,93,93.94,0.94,88,87.19,0.81
70,NYL,NYL,MIN,79,78.59,0.41,88,88.27,0.27
71,NYL,NYL,DAL,94,90.06,3.94,74,69.89,4.11


In [41]:
predictions_df[predictions_df["Model"] == "PHO"]

,Model,team,opp,team_score,team_score_pred,team_score_mae,opp_score,opp_score_pred,opp_score_mae
72,PHO,PHO,DAL,104,94.01,9.99,96,89.23,6.77
73,PHO,PHO,LAS,92,91.15,0.85,78,87.88,9.88
74,PHO,PHO,MIN,60,65.03,5.03,73,85.12,12.12
75,PHO,PHO,IND,89,87.53,1.47,98,89.38,8.62
76,PHO,PHO,DAL,92,89.43,2.57,107,91.79,15.21
77,PHO,PHO,LVA,99,89.37,9.63,103,90.46,12.54
78,PHO,PHO,CHI,93,91.70,1.30,88,90.24,2.24
79,PHO,PHO,CHI,86,86.01,0.01,68,69.58,1.58


In [42]:
predictions_df[predictions_df["Model"] == "SEA"]

,Model,team,opp,team_score,team_score_pred,team_score_mae,opp_score,opp_score_pred,opp_score_mae
80,SEA,SEA,CHI,84,84.72,0.72,88,84.50,3.50
81,SEA,SEA,IND,89,90.77,1.77,77,83.11,6.11
82,SEA,SEA,CON,72,78.73,6.73,61,67.37,6.37
83,SEA,SEA,IND,75,77.65,2.65,92,86.99,5.01
84,SEA,SEA,IND,85,87.00,2.00,83,84.61,1.61
85,SEA,SEA,DAL,92,94.40,2.40,84,85.27,1.27
86,SEA,SEA,LAS,90,89.19,0.81,87,84.51,2.49
87,SEA,SEA,WAS,83,78.65,4.35,77,69.70,7.30


In [43]:
predictions_df[predictions_df["Model"] == "WAS"]

,Model,team,opp,team_score,team_score_pred,team_score_mae,opp_score,opp_score_pred,opp_score_mae
88,WAS,WAS,LAS,82,86.93,4.93,80,84.40,4.40
89,WAS,WAS,DAL,92,87.69,4.31,84,89.46,5.46
90,WAS,WAS,DAL,97,88.12,8.88,69,75.45,6.45
91,WAS,WAS,MIN,83,86.04,3.04,99,94.83,4.17
92,WAS,WAS,PHO,80,78.07,1.93,83,81.66,1.34
93,WAS,WAS,ATL,87,85.25,1.75,68,71.83,3.83
94,WAS,WAS,ATL,73,68.81,4.19,76,76.11,0.11
95,WAS,WAS,SEA,77,68.72,8.28,83,75.70,7.30


In [44]:
predictions_df[predictions_df["Model"] == "League"]

,Model,team,opp,team_score,team_score_pred,team_score_mae,opp_score,opp_score_pred,opp_score_mae
96,League,CHI,LAS,92,91.83,0.17,78,78.61,0.61
97,League,SEA,LVA,83,82.09,0.91,94,92.12,1.88
98,League,PHO,WAS,77,77.64,0.64,90,91.23,1.23
99,League,LVA,CON,84,85.47,1.47,71,71.66,0.66
100,League,PHO,SEA,70,69.60,0.40,89,89.75,0.75
...,...,...,...,...,...,...,...,...,...
187,League,LAS,DAL,87,91.14,4.14,81,81.20,0.20
188,League,LAS,SEA,87,86.90,0.10,90,89.13,0.87
189,League,SEA,PHO,80,81.16,1.16,62,58.96,3.04
190,League,ATL,CON,82,83.33,1.33,70,69.86,0.14


In [45]:
mae_summary = (
    predictions_df
    .groupby("Model")[["team_score_mae", "opp_score_mae"]]
    .agg(["min", "max", "mean", "median"])
    .reset_index()
)

# optional: flatten column names
mae_summary.columns = ['_'.join(col).strip('_') for col in mae_summary.columns]

mae_summary

,Model,team_score_mae_min,team_score_mae_max,team_score_mae_mean,team_score_mae_median,opp_score_mae_min,opp_score_mae_max,opp_score_mae_mean,opp_score_mae_median
0,ATL,0.03,13.73,5.88500,5.220,0.23,8.80,5.217500,5.880
1,CHI,1.28,5.91,2.99000,2.320,1.00,7.99,3.203750,2.710
2,CON,0.87,9.40,4.68375,4.450,0.43,8.72,3.190000,2.435
3,DAL,0.20,3.54,1.62875,1.470,0.63,5.58,2.477500,1.720
4,IND,0.47,8.40,2.50875,1.770,0.35,9.13,3.357500,2.920
5,LAS,0.74,7.83,3.78875,3.025,0.81,5.53,2.566250,2.195
6,LVA,0.44,8.55,3.75375,3.195,1.73,9.98,5.005000,4.815
7,League,0.04,19.78,2.10000,1.290,0.03,19.72,1.804271,0.955
8,MIN,0.58,9.59,4.72875,4.515,1.93,10.58,3.820000,3.165
9,NYL,0.02,3.94,0.87500,0.480,0.27,5.04,2.275000,2.055


In [50]:
# store importances for all models
all_importances = []

for team_name in teams:
    # subset data
    if team_name == "League":
        df_team = df.copy()
    else:
        df_team = df[df["team"] == team_name].copy()

    # features
    X = df_team.drop(columns=["team", "opp", "team_score", "opp_score"])
    y = df_team[["team_score", "opp_score"]]

    # train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # model
    model = MultiOutputRegressor(RandomForestRegressor(random_state=42))
    model.fit(X_train, y_train)

    # get feature importances from both estimators
    team_score_importance = model.estimators_[0].feature_importances_
    opp_score_importance = model.estimators_[1].feature_importances_

    # stack into rows
    for i, feature in enumerate(X.columns):
        all_importances.append({
            "Model": team_name,
            "Feature": feature,
            "Target": "team_score",
            "Importance": team_score_importance[i]
        })
        all_importances.append({
            "Model": team_name,
            "Feature": feature,
            "Target": "opp_score",
            "Importance": opp_score_importance[i]
        })

# build final DataFrame
importances_df = pd.DataFrame(all_importances)

# optional: sort by importance descending
importances_df = importances_df.sort_values(by="Importance", ascending=True)

importances_df.head(20)

,Model,Feature,Target,Importance
753,League,win_loss,opp_score,0.000087
637,SEA,win_loss,opp_score,0.000093
636,SEA,win_loss,team_score,0.000139
695,WAS,win_loss,opp_score,0.000157
230,DAL,win_loss,team_score,0.000157
173,CON,win_loss,opp_score,0.000178
288,IND,win_loss,team_score,0.000193
557,PHO,month_by_team_day_of_week,opp_score,0.000225
115,CHI,win_loss,opp_score,0.000231
521,NYL,win_loss,opp_score,0.000251


In [53]:
# group by feature and target, compute min and max
importance_range = (
    importances_df
    .groupby(["Feature", "Target"])["Importance"]
    .agg(["min", "max"])
    .reset_index()
)

# pivot so columns are like: team_score_min, team_score_max, etc.
importance_summary = importance_range.pivot(index="Feature", columns="Target")
importance_summary.columns = [f"{target}_{stat}" for stat, target in importance_summary.columns]
importance_summary = importance_summary.reset_index()

In [55]:
# reorder columns
importance_summary = importance_summary[
    ["Feature", "team_score_min", "team_score_max", "opp_score_min", "opp_score_max"]
]

importance_summary

,Feature,team_score_min,team_score_max,opp_score_min,opp_score_max
0,advanced_3par,0.001368,0.015632,0.001566,0.030952
1,advanced_ast_pct,0.001177,0.013680,0.001466,0.033590
2,advanced_blk_pct,0.001692,0.010159,0.001031,0.015042
3,advanced_drtg,0.001645,0.022946,0.212673,0.840354
4,advanced_ftr,0.001292,0.018846,0.001580,0.037911
5,advanced_ortg,0.197502,0.789965,0.001261,0.036461
6,advanced_pace,0.004264,0.096838,0.002939,0.085920
7,advanced_pace_by_team_day_of_week,0.001982,0.006983,0.001578,0.018228
8,advanced_stl_pct,0.002085,0.011873,0.003171,0.036225
9,advanced_trb_pct,0.001663,0.020075,0.001343,0.029437


In [48]:
predictions_df.to_csv("wnba_predictions_v1.csv", index=False)
results_df.to_csv("wnba_model_summary_v1.csv", index=False)


files.download("wnba_predictions_v1.csv")
files.download("wnba_model_summary_v1.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>